In [1]:
import pandas as pd
import pandas_ta as ta
from backtesting import Strategy
from backtesting.lib import barssince
import backtesting.lib as bt
import datetime as dt

/Users/AmirPurified/opt/anaconda3/envs/ml8/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [57]:
data = pd.read_csv("/Users/AmirPurified/Documents/Work/crypto-data-analysis/data/test_2021/extracted_data/concated.csv", index_col=False)
data = data[["Open","High","Low","Close","Date"]]
data.Date= pd.to_datetime(data.Date)
data.Date = data.Date.apply(lambda x: dt.datetime.strftime(x, '%Y-%m-%d %H:%M:%S'))
data = data.drop_duplicates()
data.to_csv('cleaned_data.csv', index=False)
data


,Open,High,Low,Close,Date
0,28923.63,29600.00,28624.57,29331.69,2021-01-01 00:00:00
1,29331.70,33300.00,28946.53,32178.33,2021-01-02 00:00:00
2,32176.45,34778.11,31962.99,33000.05,2021-01-03 00:00:00
3,33000.05,33600.00,28130.00,31988.71,2021-01-04 00:00:00
4,31989.75,34360.00,29900.00,33949.53,2021-01-05 00:00:00
...,...,...,...,...,...
1518,23242.42,23452.00,22665.85,22963.00,2023-02-08 00:00:00
1520,22961.85,23011.39,21688.00,21796.35,2023-02-09 00:00:00
1522,21797.83,21938.16,21451.00,21625.19,2023-02-10 00:00:00
1524,21625.19,21906.32,21599.78,21862.55,2023-02-11 00:00:00


In [58]:
data = pd.read_csv("./cleaned_data.csv", index_col="Date", parse_dates=True)
# data = data.set_index("Date")
data.index.name = None
# data.index= pd.to_datetime(data.index)
data

,Open,High,Low,Close
2021-01-01,28923.63,29600.00,28624.57,29331.69
2021-01-02,29331.70,33300.00,28946.53,32178.33
2021-01-03,32176.45,34778.11,31962.99,33000.05
2021-01-04,33000.05,33600.00,28130.00,31988.71
2021-01-05,31989.75,34360.00,29900.00,33949.53
...,...,...,...,...
2023-02-08,23242.42,23452.00,22665.85,22963.00
2023-02-09,22961.85,23011.39,21688.00,21796.35
2023-02-10,21797.83,21938.16,21451.00,21625.19
2023-02-11,21625.19,21906.32,21599.78,21862.55


In [59]:
ta.sma(data.High).info()

<class 'pandas.core.series.Series'>
DatetimeIndex: 764 entries, 2021-01-01 to 2023-02-12
Series name: SMA_10
Non-Null Count  Dtype  
--------------  -----  
755 non-null    float64
dtypes: float64(1)
memory usage: 11.9 KB


In [60]:
b=c=d=5
a=5
if a==5\
    and b==5:
    print ("met")

met


In [148]:
class SmaConsec(Strategy):
    # Define the two MA lags as *class variables*
    # for later optimization
    n1 = 10
    n2 = 10
    
    def init(self):
        # Precompute the two moving averages
        self.sma_h = self.I(ta.sma, pd.Series(self.data.High), self.n1)
        self.sma_l = self.I(ta.sma, pd.Series(self.data.Low), self.n2)
        self.consec_high_candl=1
        self.consec_low_candl=1
    
    def next(self):
        price = self.data.Close[-1]
        # print (self.consec_high_candl)
        # print (self.data.High[-1],self.sma_h[-1])

        if self.data.High[-1]>=self.sma_h[-1]:
            self.consec_high_candl += 1
            if self.consec_high_candl == 5:
                self.buy(tp=1.04*price, sl=0.98*price)
                self.consec_high_candl=1
        else:
            self.consec_high_candl=1

        if self.data.Low[-1]<=self.sma_l[-1]:
            self.consec_low_candl += 1
            if self.consec_low_candl == 5:
                self.sell(tp=0.96*price, sl=1.02*price)
                self.consec_low_candl=1
        else:
            self.consec_low_candl=1

        # print (self.orders)
        
        # if barssince(self.data.High>=self.sma_h)==5:
        #     and barssince(self.data.Open>=self.sma_h)==5\
        #     and barssince(self.data.Close>=self.sma_h)==5\
        #     and barssince(self.data.Low>=self.sma_h)==5:
        #     #self.position.close()
        #     self.buy(tp=1.04*price, sl=0.98*price)
            
            
        # elif barssince(self.data.Low<=self.sma_l)==5:
        #     and barssince(self.data.Open<=self.sma_l)==5\
        #     and barssince(self.data.Close<=self.sma_l)==5\
        #     and barssince(self.data.Low<=self.sma_l)==5:
        #     self.position.close()
        #     self.sell(tp=0.94*price, sl=1.02*price)

        # Else, if sma1 crosses below sma2, close any existing
        # long trades, and sell the asset
        # elif crossover(self.sma2, self.sma1):
        #     self.position.close()
        #     self.sell()

In [157]:
from backtesting import Backtest

bt = Backtest(data, SmaConsec, cash=10000000, commission=.001)
stats = bt.run()
print (stats)

Start                     2021-01-01 00:00:00
End                       2023-02-12 00:00:00
Duration                    772 days 00:00:00
Exposure Time [%]                    25.91623
Equity Final [$]                3937528.39869
Equity Peak [$]                10521987.25013
Return [%]                         -60.624716
Buy & Hold Return [%]              -25.733771
Return (Ann.) [%]                  -35.935321
Volatility (Ann.) [%]               14.044871
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -64.068472
Avg. Drawdown [%]                  -23.442721
Max. Drawdown Duration      725 days 00:00:00
Avg. Drawdown Duration      248 days 00:00:00
# Trades                                  129
Win Rate [%]                        23.255814
Best Trade [%]                        3.92487
Worst Trade [%]                     -3.545127
Avg. Trade [%]                    

In [150]:
import backtesting
backtesting.set_bokeh_output(notebook=False)

In [151]:
bt.plot()

Row(id='14308', ...)

In [154]:
print(stats['_trades'].to_string())

     Size  EntryBar  ExitBar   EntryPrice   ExitPrice           PnL  ReturnPct  EntryTime   ExitTime Duration
0    -312        23       23  32045.92200  32719.5600 -210175.05600  -0.021021 2021-01-24 2021-01-24   0 days
1    -322        27       27  30331.82781  30973.4730 -206609.75118  -0.021154 2021-01-28 2021-01-28   0 days
2     285        32       32  33550.60709  34867.4248  375293.04735   0.039249 2021-02-02 2021-02-02   0 days
3     259        36       36  38327.60932  39821.8496  387008.23252   0.038986 2021-02-06 2021-02-06   0 days
4     222        40       40  46466.84042  45492.0116 -216411.99804  -0.020979 2021-02-10 2021-02-10   0 days
5     214        44       44  47203.93678  49039.8376  392882.77548   0.038893 2021-02-14 2021-02-14   0 days
6     201        48       48  52169.78767  51077.3158 -219586.84587  -0.020941 2021-02-18 2021-02-18   0 days
7     179        52       52  57469.76235  56260.3986 -216476.11125  -0.021043 2021-02-22 2021-02-22   0 days
8    -214 

In [156]:
stats['_trades'].to_csv("trade_list.csv")